In [3]:
import tensorflow as tf
import numpy as np
from gensim.models.keyedvectors import KeyedVectors

In [4]:
import re

In [5]:
glove = KeyedVectors.load_word2vec_format("glove.6B.50d.txt.w2v", binary=False) # Slow

In [32]:
print(glove["hello"])
print(glove["world"])
print(glove["arent"])

[-0.38497   0.80092   0.064106 -0.28355  -0.026759 -0.34532  -0.64253
 -0.11729  -0.33257   0.55243  -0.087813  0.9035    0.47102   0.56657
  0.6985   -0.35229  -0.86542   0.90573   0.03576  -0.071705 -0.12327
  0.54923   0.47005   0.35572   1.2611   -0.67581  -0.94983   0.68666
  0.3871   -1.3492    0.63512   0.46416  -0.48814   0.83827  -0.9246
 -0.33722   0.53741  -1.0616   -0.081403 -0.67111   0.30923  -0.3923
 -0.55002  -0.68827   0.58049  -0.11626   0.013139 -0.57654   0.048833
  0.67204 ]
[-0.41486   0.71848  -0.3045    0.87445   0.22441  -0.56488  -0.37566
 -0.44801   0.61347  -0.11359   0.74556  -0.10598  -1.1882    0.50974
  1.3511    0.069851  0.73314   0.26773  -1.1787   -0.148     0.039853
  0.033107 -0.27406   0.25125   0.41507  -1.6188   -0.81778  -0.73892
 -0.28997   0.57277   3.4719    0.73817  -0.044495 -0.15119  -0.93503
 -0.13152  -0.28562   0.76327  -0.83332  -0.6793   -0.39099  -0.64466
  1.0044   -0.2051    0.46799   0.99314  -0.16221  -0.46022  -0.37639
 -0.6754

In [7]:
from tensorflow.keras import layers

In [8]:
def encode(text):
    x = []
    text = text.lower()
    
    words = text.split()
    for word in words:
        word = re.sub(r'[^a-zA-Z]', "", word)
        try:
            x.append(glove[word])
        except:
            print()
    return x

In [9]:
encode("Hello World")

[array([-0.38497 ,  0.80092 ,  0.064106, -0.28355 , -0.026759, -0.34532 ,
        -0.64253 , -0.11729 , -0.33257 ,  0.55243 , -0.087813,  0.9035  ,
         0.47102 ,  0.56657 ,  0.6985  , -0.35229 , -0.86542 ,  0.90573 ,
         0.03576 , -0.071705, -0.12327 ,  0.54923 ,  0.47005 ,  0.35572 ,
         1.2611  , -0.67581 , -0.94983 ,  0.68666 ,  0.3871  , -1.3492  ,
         0.63512 ,  0.46416 , -0.48814 ,  0.83827 , -0.9246  , -0.33722 ,
         0.53741 , -1.0616  , -0.081403, -0.67111 ,  0.30923 , -0.3923  ,
        -0.55002 , -0.68827 ,  0.58049 , -0.11626 ,  0.013139, -0.57654 ,
         0.048833,  0.67204 ], dtype=float32),
 array([-0.41486 ,  0.71848 , -0.3045  ,  0.87445 ,  0.22441 , -0.56488 ,
        -0.37566 , -0.44801 ,  0.61347 , -0.11359 ,  0.74556 , -0.10598 ,
        -1.1882  ,  0.50974 ,  1.3511  ,  0.069851,  0.73314 ,  0.26773 ,
        -1.1787  , -0.148   ,  0.039853,  0.033107, -0.27406 ,  0.25125 ,
         0.41507 , -1.6188  , -0.81778 , -0.73892 , -0.28997 ,  0

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(50, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [40]:
model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [41]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [42]:
def sample_predict(sentence, pad):
    encoded_sample_pred_text = encode(sample_pred_text)
    print(encoded_sample_pred_text)
    print("len: "+str(len(encoded_sample_pred_text)))
    print("shape: "+str(encoded_sample_pred_text[0].shape))
#     encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64)
    encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
    print("Encodedsamplepredtext shape: "+str(encoded_sample_pred_text)) # shape is (19,50)
    predictions = model.predict(encoded_sample_pred_text) # This is definitely where it fails and no one knows why
    print(predictions)
    return (predictions)

In [43]:
sample_pred_text = ('The movie was cool. The animation and the graphics '
                    'were out of this world. I would recommend this movie.')
predictions = sample_predict(sample_pred_text, pad=False)
print (predictions)

[array([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
       -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
        2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
        1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
       -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
       -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
        4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
        7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
       -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
        1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01],
      dtype=float32), array([ 0.30824 ,  0.17223 , -0.23339 ,  0.023105,  0.28522 ,  0.23076 ,
       -0.41048 , -1.0035  , -0.2072  ,  1.4327  , -0.80684 ,  0.68954 ,
       -0.43648 ,  1.1069  ,  1.6107  , -0.31966 ,  0.47744 ,  0.79395 ,
       -0.84374 ,  0.064509,  0.90251

ValueError: Input 0 of layer sequential_2 is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: [None, 50]

In [45]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
    pass
import tensorflow_datasets as tfds
import tensorflow as tf
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

In [46]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

In [48]:
encoder = info.features['text'].encoder
print ('Vocabulary size: {}'.format(encoder.vocab_size))


Vocabulary size: 8185


In [57]:
encoded_string = encoder.encode('The movie was cool. The andrew seven yang loser mouse giraffe zebra audfsfdsstonomous pie')
for index in encoded_string:
    print('{} ----> {}'.format(index, encoder.decode([index])))

19 ----> The 
27 ----> movie 
18 ----> was 
2724 ----> cool
3 ----> . 
19 ----> The 
206 ----> and
7137 ----> rew
7961 ---->  
6562 ----> seven 
7381 ----> yan
291 ----> g 
6366 ----> loser
7961 ---->  
6408 ----> mous
49 ----> e 
786 ----> gi
670 ----> ra
3053 ----> ffe
7961 ---->  
1270 ----> ze
1861 ----> bra
7961 ---->  
1143 ----> au
8029 ----> d
8031 ----> f
3898 ----> sf
1761 ----> ds
7386 ----> ston
795 ----> om
339 ----> ous 
2972 ----> pie


In [58]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE, train_dataset.output_shapes)

test_dataset = test_dataset.padded_batch(BATCH_SIZE, test_dataset.output_shapes)

In [59]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [60]:
def pad_to_size(vec, size):
    zeros = [0] * (size - len(vec))
    vec.extend(zeros)
    return vec

In [61]:
def sample_predict(sentence, pad):
    encoded_sample_pred_text = encoder.encode(sample_pred_text)

    if pad:
        encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64)
    encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
    predictions = model.predict(tf.expand_dims(encoded_sample_pred_text, 0))


    return (predictions)

In [63]:
# predict on a sample text without padding.

sample_pred_text = ('I think Alexander C. Sun is really cool')
predictions = sample_predict(sample_pred_text, pad=False)
print (predictions)

tf.Tensor(
[1.200e+01 1.290e+02 5.834e+03 1.250e+02 7.996e+03 3.000e+00 3.778e+03
 7.961e+03 9.000e+00 8.100e+01 2.724e+03], shape=(11,), dtype=float32)
tf.Tensor(
[[1.200e+01 1.290e+02 5.834e+03 1.250e+02 7.996e+03 3.000e+00 3.778e+03
  7.961e+03 9.000e+00 8.100e+01 2.724e+03]], shape=(1, 11), dtype=float32)
[[0.5018966]]
